# Lab session of 23/03/2022

## Graph problem 2: Minimum Spanning Tree, cutset formulation

Write and solve the cut-set formulation of the Minimum Spanning Tree (MST) for a random graph.

## Solution

A formulation of the MST problem uses *cut sets*: given a graph $G=(V,E)$ with $V$ the set of nodes and $E$ the set of edges, we create binary variables $x_{ij}$ for each $(i,j)\in E$ such that $x_{ij}$ is 1 if $(i,j)$ is in the solution, 0 otherwise. Then the formulation includes $cut set$ inequalities.

A *cut* $\delta(S)$ in a graph $G=(V,E)$ is a subset of edges generated by a subset of nodes $S\subset V$: an edge $(i,j)\in E$ is in $\delta(S)$ if exactly one among $i$ and $j$ is in $S$.


The formulation enforces the following condition:

__For any cut $\delta(S)$ containing the set of edges $(i,j)$ with $i$ in $S\subset V$ and $j$ in $V\setminus S$, there must be at least one edge of $\delta(S)$ in the solution.__

This implies that a formulation for the MST is 

$$
\begin{array}{lll}
  \min & \sum_{(i,j)\in E} c_{ij} x_{ij}\\
  \textrm{s.t.} & \sum_{(i,j)\in \mathcal \delta(S)} x_{ij} \ge 1 & \forall S\subset V:S\neq \emptyset\\
                & x_{ij} \in \{0,1\} & \forall (i,j)\in E
\end{array}
$$

Write the above formulation and solve it for a random graph.

In [1]:
# When using Colab, make sure you run this instruction beforehand
!pip install mip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 19.6 MB/s eta 0:00:00


In [5]:
import numpy as np
import math

k = 11 #nodi
grid_size = 100 # size of the grid of points

# distance beyond which we don't want edges
d_max = .75 * grid_size

np.random.seed(12345) 


# Create k random points with two coordinates. Multiplying by grid_size yields
# random numbers between 0 and 100.
point = np.random.random((k,2)) * grid_size
print(point)
# Define the set of vertices of the graph as the list of numbers from 0 to k-1
V = [i for i in range(k)]
print(V)
# Determine the distance between each point
distance = np.array([[math.sqrt(np.sum((point[i] - point[j])**2)) for i in V] for j in V])
# Set of edges: note the condition that i<j (so we have pairs i,j but not j,i)
# and especially the condition that prevents long arcs.
E= [(i,j) for i in V for j in V if i<j and distance[i,j]<= d_max]
print(E)

[[92.96160928 31.63755546]
 [18.39188117 20.45602786]
 [56.77250291 59.5544703 ]
 [96.45145197 65.31770969]
 [74.89066375 65.35698709]
 [74.77148093 96.13067361]
 [ 0.83882979 10.64443767]
 [29.87037138 65.64111831]
 [80.98125525 87.21759137]
 [96.46475974 72.36853469]
 [64.24753279 71.74536208]]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[(0, 2), (0, 3), (0, 4), (0, 5), (0, 7), (0, 8), (0, 9), (0, 10), (1, 2), (1, 4), (1, 6), (1, 7), (1, 10), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (3, 4), (3, 5), (3, 7), (3, 8), (3, 9), (3, 10), (4, 5), (4, 7), (4, 8), (4, 9), (4, 10), (5, 7), (5, 8), (5, 9), (5, 10), (6, 7), (7, 8), (7, 9), (7, 10), (8, 9), (8, 10), (9, 10)]


Below is code for generating all subsets of a given list of objects. We will use `powerset` to create all constraints of the formulation.

In [6]:
from itertools import chain, combinations
powerset = list(chain.from_iterable(combinations(V, r) for r in V))
print(powerset)

[(), (0,), (1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,), (9,), (10,), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (4, 10), (5, 6), (5, 7), (5, 8), (5, 9), (5, 10), (6, 7), (6, 8), (6, 9), (6, 10), (7, 8), (7, 9), (7, 10), (8, 9), (8, 10), (9, 10), (0, 1, 2), (0, 1, 3), (0, 1, 4), (0, 1, 5), (0, 1, 6), (0, 1, 7), (0, 1, 8), (0, 1, 9), (0, 1, 10), (0, 2, 3), (0, 2, 4), (0, 2, 5), (0, 2, 6), (0, 2, 7), (0, 2, 8), (0, 2, 9), (0, 2, 10), (0, 3, 4), (0, 3, 5), (0, 3, 6), (0, 3, 7), (0, 3, 8), (0, 3, 9), (0, 3, 10), (0, 4, 5), (0, 4, 6), (0, 4, 7), (0, 4, 8), (0, 4, 9), (0, 4, 10), (0, 5, 6), (0, 5, 7), (0, 5, 8), (0, 5, 9), (0, 5, 10), (0, 6, 7), (0, 6, 8), (0, 6, 9), (0, 6, 10), (0, 7, 8), (0, 7, 9), (0, 7, 10), (0, 8, 9)

We have all we need to write the problem.

In [ ]:
import networkx as nx

def draw_solution(V, A, x):
    g = nx.Graph()

    # Draw the whole graph first: all nodes, all arcs, no highlighting
    g.add_nodes_from(V)
    g.add_edges_from([(i,j) for (i,j) in A])
    nx.draw(g, pos=point)

    # Reset the graph and add only the arcs that belong to the solution, 
    # i.e. those for which the optimal value of the variable f[i,j] is nonzero
    g.clear()
    g.add_edges_from([(i,j) for (i,j) in A if x[i,j].x > 0.001])
    nx.draw(g, pos=point, width=4, edge_color='red')

    # finally, draw a graph consisting of the sole root node, highlighted in green
    g.clear()
    g.add_node(0)
    nx.draw(g, pos={0: point[0]}, node_color='green')
    
# after defining the function, call it with the current data
draw_solution (V, E, x)